In [204]:
import numpy as np
import pandas as pd
from datetime import date
from matplotlib import pyplot as plt

from numpy import cos, sin, arcsin, sqrt
from math import radians
import seaborn as sns
from jupyter_dash import JupyterDash

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.figure_factory as ff
import plotly.express as px
import numpy as np

# need for token (mapbox)
px.set_mapbox_access_token("pk.eyJ1IjoiZmlsaXBrcmFzbmlxaSIsImEiOiJja2luOW9jdmgwa3J3MnpvNXhkNGJ6MWFtIn0.eevoM5byqvtc1nC0oXpuOw")

def haversine(row, lonlat):
    lat1, lon1 = lonlat
    lon2, lat2 = row['LNG'], row['LAT']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

def cell_from_coords(data, lonlat):
    data['LAT'], data['LNG'] = pd.to_numeric(data['LAT_Y'],errors='coerce'), pd.to_numeric(data['LONG_X'],errors='coerce')
    data['distance'] = data.apply(lambda row: haversine(row, lonlat), axis=1)
    row = data.sort_values(by='distance').iloc[0:1]
    data.drop(['distance', 'LAT', "LNG"], axis=1, inplace=True)
    return row

def normalize(data, column):
    data.loc[:, column] = (data[column]-data[column].mean())/data[column].std()
    return data

def fix_coords(data):
    data['LAT'], data['LNG'] = pd.to_numeric(data['LAT_Y'],errors='coerce'), pd.to_numeric(data['LONG_X'],errors='coerce')
    return data

def prepare_for_hexbin(dataframe, weekly = True):
    if weekly:
        data_groupped = dataframe.groupby('ECELL_ID').resample('W-Mon', on='Date').mean().reset_index().sort_values(by='Date')
    else:
        data_groupped = dataframe.groupby('ECELL_ID').resample('MS', on='Date').mean().reset_index().sort_values(by='Date') 
    data_groupped = data_groupped.sort_values(by='Date')
    data_groupped['DateString'] = data_groupped['Date'].map(lambda x: x.strftime('%j'))
    data_groupped["DateString"] = (data_groupped["DateString"].astype(int)+1)//7
    return data_groupped

def remove_outliers(df, col):
    df = df.dropna(subset=[col])
    return df[df[col] < np.percentile(df[col],95)]


data_path = "/Users/filipkrasniqi/Documents/Datasets.tmp/traffic-covid/"
cities = ["Milano", "ROMA", "TORINO"]

# Reading data regarding Rome and Milan

In [174]:
#data_milano, data_rome, data_turin = [pd.read_pickle("{}LTE_1800_{}.pkl".format(data_path, city)) for city in cities]
#data_milano, data_rome, data_turin = fix_coords(data_milano), fix_coords(data_rome), fix_coords(data_turin)
data_milano = fix_coords(pd.read_pickle("{}LTE_1800_{}.pkl".format(data_path, cities[0])))
data_milano_2 = fix_coords(pd.read_pickle("{}LTE_1800_{}_P2.pkl".format(data_path, cities[0])))
data_rome = fix_coords(pd.read_pickle("{}LTE_1800_{}.pkl".format(data_path, cities[1])))

In [206]:
data_milano = fix_coords(pd.read_pickle("{}LTE_1800_{}.pkl".format(data_path, cities[0])))

In [208]:
# remove outliers from the three datasets regarding DL_VOL and Hin_Succ
data_milano = remove_outliers(data_milano, "DL_VOL")
data_milano = remove_outliers(data_milano, "Hin_Succ")

data_milano_2 = remove_outliers(data_milano_2, "DL_VOL")
data_milano_2 = remove_outliers(data_milano_2, "Hin_Succ")
#data_rome = remove_outliers(data_rome, "DL_VOL")
data_rome = remove_outliers(data_rome, "DL_VOL")
data_rome = remove_outliers(data_rome, "Hin_Succ")

In [209]:
# merging them
all_data_milano = pd.concat([data_milano, data_milano_2])
all_data_milano["City"] = 0
data_rome["City"] = 1
all_data = pd.concat([all_data_milano, data_rome])
all_data["USERNUM_AVG"] = all_data["USERNUM_AVG"].astype(float)

In [210]:
data_groupped_week = prepare_for_hexbin(all_data)

In [211]:
data_groupped_month = prepare_for_hexbin(all_data, weekly = False)

In [212]:
columns, required_columns = ["DL_VOL", "Hin_Succ", "USERNUM_AVG"], ["LAT", "LNG", "DateString", "City"]
week_selected_data_groupped = data_groupped_week[columns+required_columns].dropna()
#selected_data_groupped = data_groupped[columns+required_columns].dropna()
week_selected_data_groupped["City"] = week_selected_data_groupped["City"].astype(int)
week_selected_data_groupped["City"] = week_selected_data_groupped["City"].apply(lambda x: cities[x])

month_selected_data_groupped = data_groupped_month[columns+required_columns].dropna()
#selected_data_groupped = data_groupped[columns+required_columns].dropna()
month_selected_data_groupped["City"] = month_selected_data_groupped["City"].astype(int)
month_selected_data_groupped["City"] = month_selected_data_groupped["City"].apply(lambda x: cities[x])

In [230]:
app = JupyterDash("Map")

app.layout = html.Div([
    html.Label(["KPIs", dcc.Dropdown(
        id="kpi",
        options=[{"label": x, "value": x} for x in columns],
        value=columns[0],
        clearable=False,
                )]),
    html.Label(
        [
            "Città",
            dcc.Dropdown(id="city",
                         options=[{"label": x, "value": x} for x in week_selected_data_groupped.City.unique()],
                        value=week_selected_data_groupped["City"].unique()[1],
                        clearable=False,),
        ]
    ),
    html.Label(
        [
            "Data range",
            dcc.Dropdown(id="dateRange",
                         options=[{"label": x, "value": x} for x in ["Week", "Month"]],
                        value="Week",
                        clearable=False,),
        ]
    ),
    html.Div(dcc.Graph(id="map-chart"))
]) # , style={'columnCount': 2}

@app.callback(
    Output("map-chart", "figure"), 
    [Input("kpi", "value"), Input("city", "value"), Input("dateRange", "value")])

def display_map(kpi, city, dateRange):
    
    if dateRange == "Week":
        selected_data_groupped = week_selected_data_groupped
    else:
        selected_data_groupped = month_selected_data_groupped
    
    filtered_data_groupped = selected_data_groupped.where(lambda x:x.City==city).dropna()
    fig = ff.create_hexbin_mapbox(
        data_frame=filtered_data_groupped,
        lat="LAT", lon="LNG", nx_hexagon=30, animation_frame="DateString", color=kpi,
        color_continuous_scale="Inferno", labels={"color": kpi, "frame": "DateString"}
    )
    fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
    fig.layout.sliders[0].pad.t=20
    fig.layout.updatemenus[0].pad.t=60

#     fig.show()  
    return fig

app.run_server(mode='inline') # debug=True, use_reloader=False

In [215]:
# preparing data for period visualization
all_data["hour"] = [t.hour for t in pd.DatetimeIndex(all_data.Date)]
columns_all_data, required_columns_all_data = ["DL_VOL", "Hin_Succ", "USERNUM_AVG"], ["LAT", "LNG", "City", 'hour', 'Date', 'ECELL_ID']
periods = ["Dawn", "Morning", "Lunch", "Afternoon", "Evening"]
# getting data with relative period
evening_data = all_data[columns_all_data+required_columns_all_data].where(lambda x: (x.hour >= 20) & (x.hour <= 23)).dropna()
afternoon_data = all_data[columns_all_data+required_columns_all_data].where(lambda x: (x.hour >= 16) & (x.hour <= 19)).dropna()
lunch_data = all_data[columns_all_data+required_columns_all_data].where(lambda x: (x.hour >= 12) & (x.hour <= 15)).dropna()
morning_data = all_data[columns_all_data+required_columns_all_data].where(lambda x: (x.hour >= 8) & (x.hour <= 11)).dropna()
dawn_data = all_data[columns_all_data+required_columns_all_data].where(lambda x: (x.hour >= 4) & (x.hour <= 7)).dropna()
# assign column "period"
evening_data["period"] = 4
afternoon_data["period"] = 3
lunch_data["period"] = 2
morning_data["period"] = 1
dawn_data["period"] = 0
# TODO io voglio un punto per ogni (cella, periodo): è così??? in teoria no!!
# merge all in one

evening_data_week, afternoon_data_week, lunch_data_week, morning_data_week, dawn_data_week = \
        prepare_for_hexbin(evening_data), prepare_for_hexbin(afternoon_data), prepare_for_hexbin(lunch_data), \
        prepare_for_hexbin(morning_data), prepare_for_hexbin(dawn_data)

evening_data_month, afternoon_data_month, lunch_data_month, morning_data_month, dawn_data_month = \
        prepare_for_hexbin(evening_data, weekly = False), prepare_for_hexbin(afternoon_data, weekly = False), \
        prepare_for_hexbin(lunch_data, weekly = False), prepare_for_hexbin(morning_data, weekly = False), \
        prepare_for_hexbin(dawn_data, weekly = False)

#period_data_groupped_month = prepare_for_hexbin(period_data, weekly = False)
#period_data = pd.concat([dawn_data, morning_data, lunch_data, afternoon_data, evening_data])
period_data_groupped_week = pd.concat([evening_data_week, afternoon_data_week, lunch_data_week, morning_data_week, dawn_data_week])
period_data_groupped_month = pd.concat([evening_data_month, afternoon_data_month, lunch_data_month, morning_data_month, dawn_data_month])

period_columns = columns+required_columns+['period', 'ECELL_ID']
period_week_selected_data_groupped = period_data_groupped_week[period_columns].dropna()
period_week_selected_data_groupped["period"] = period_week_selected_data_groupped["period"].astype(int)
period_week_selected_data_groupped["City"] = period_week_selected_data_groupped["City"].astype(int)
period_week_selected_data_groupped["City"] = period_week_selected_data_groupped["City"].apply(lambda x: cities[x])
period_week_selected_data_groupped["period"] = period_week_selected_data_groupped["period"].apply(lambda x: periods[x])

period_month_selected_data_groupped = period_data_groupped_month[period_columns].dropna()
period_month_selected_data_groupped["period"] = period_month_selected_data_groupped["period"].astype(int)
period_month_selected_data_groupped["City"] = period_month_selected_data_groupped["City"].astype(int)
period_month_selected_data_groupped["City"] = period_month_selected_data_groupped["City"].apply(lambda x: cities[x])
period_month_selected_data_groupped["period"] = period_month_selected_data_groupped["period"].apply(lambda x: periods[x])

In [232]:
app_period = JupyterDash("PERIOD")

app_period.layout = html.Div([
    html.Label(["KPIs", dcc.Dropdown(
        id="kpi",
        options=[{"label": x, "value": x} for x in columns],
        value=columns[0],
        clearable=False,
                )]),
    html.Label(
        [
            "Città",
            dcc.Dropdown(id="city",
                         options=[{"label": x, "value": x} for x in period_week_selected_data_groupped.City.unique()],
                        value=period_week_selected_data_groupped["City"].unique()[1],
                        clearable=False,),
        ]
    ),
    html.Label(
        [
            "Data range",
            dcc.Dropdown(id="dateRange",
                         options=[{"label": x, "value": x} for x in ["Week", "Month"]],
                        value="Week",
                        clearable=False,),
        ]
    ),
    html.Label(
        [
            "Period",
            dcc.Dropdown(id="period",
                         options=[{"label": x, "value": x} for x in period_week_selected_data_groupped.period.unique()],
                        value="Dawn",
                        clearable=False,),
        ]
    ),
    html.Label(
        [
            "Period vs evening",
            dcc.Dropdown(id="period_vs_evening",
                         options=[{"label": x, "value": x} for x in ["Yes", "No"]],
                        value="No",
                        clearable=False,),
        ]
    ),
    html.Div(dcc.Graph(id="period-chart"))])
    
@app_period.callback(
    Output("period-chart", "figure"), 
    [Input("kpi", "value"), Input("city", "value"), Input("dateRange", "value"), Input("period", "value"), Input("period_vs_evening", "value")])
def display_map_period(kpi, city, dateRange, period, periodVsEvening):
    
    if period == 'Evening':
        periodVsEvening = 'No'
    
    if dateRange == "Week":
        selected_data_groupped = period_week_selected_data_groupped
    else:
        selected_data_groupped = period_month_selected_data_groupped
    
    filtered_data_groupped = selected_data_groupped.where(lambda x:x.City==city).dropna()
    if periodVsEvening == 'No':
        filtered_data_groupped = filtered_data_groupped.where(lambda x:x.period==period).dropna()
        filtered_data_groupped = filtered_data_groupped.set_index(['ECELL_ID', 'DateString'])
        filtered_data_groupped.reset_index(inplace=True)
    else:
        filtered_data_groupped = filtered_data_groupped.set_index(['ECELL_ID', 'DateString'])
        filtered_data_groupped_evening = filtered_data_groupped.where(lambda x:x.period=='Evening').dropna()
        filtered_data_groupped_period = filtered_data_groupped.where(lambda x:x.period==period).dropna()
        
        filtered_data_groupped = filtered_data_groupped_evening#
        filtered_data_groupped[kpi] = (filtered_data_groupped[kpi] - filtered_data_groupped_period[kpi]).abs()
        
        filtered_data_groupped.reset_index(inplace=True)
        filtered_data_groupped_evening.reset_index(inplace=True)
        filtered_data_groupped_period.reset_index(inplace=True)
        
    fig = ff.create_hexbin_mapbox(
        data_frame=filtered_data_groupped,
        lat="LAT", lon="LNG", nx_hexagon=30, animation_frame="DateString", color=kpi,
        color_continuous_scale="Inferno", labels={"color": kpi, "frame": "DateString"}
    )
    fig.update_layout(margin=dict(b=20, t=20, l=0, r=0))
    
    fig.layout.sliders[0].pad.t=30
    fig.layout.updatemenus[0].pad.t=50

#     fig.show()  
    return fig

app_period.run_server(mode='inline', port=16000) # debug=True, use_reloader=False

In [ ]:
# TODO parlare con Andrea per analizzare Roma (dritte su zone e altro)
# TODO different KPIs -> Filip -> X
# TODO per month instead of week -> Filip -> X

# TODO represent in different period during days, and compare with dinner time (as a base for home) -> Filip
# TODO represent in different period of weeks, and compare with weekend -> Filip

# TODO find info about density of population on specific places / areas -> Franci, (Filip)
# TODO find area specification per area (e.g.: business, home, turistic, ...) -> Franci, (Filip)

# -> brainstorming: attempt on 23/12

# es: Garibaldi -> DoW, h8-h12, h14-h18: work
# DoW + WE, h18-6: home

# TODO ratio at every time of dl_link
# TODO dl_link/#user -> low values represent high mobility areas